In [7]:
import cv2
from ultralytics import YOLO
import uuid

print(cv2.__version__)

# Load the YOLOv8 model
model = YOLO("yolov8s.pt")  # You can choose different model sizes: yolov8n.pt, yolov8s.pt, etc.

# Open the video file
video_path = "PeopleCounting-ComputerVision/3.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

trackers = []
person_ids = set()

def create_tracker(frame, bbox):
    tracker = cv2.legacy.TrackerKCF_create()
    tracker.init(frame, bbox)
    return tracker

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform inference on the current frame
    results = model.predict(frame, device="cpu")

    # Extract bounding boxes and labels
    detections = results[0].boxes

    # Filter detections to count only 'person' class (class 0 in COCO dataset)
    for detection in detections:
        label = int(detection.cls)  # YOLO model returns label as 'cls' attribute
        if label == 0:  # COCO class 0 is 'person'
            # Extract bounding box
            bbox = detection.xyxy[0].cpu().numpy()
            x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2] - bbox[0]), int(bbox[3] - bbox[1])

            # Create a unique ID for the detected person
            person_id = str(uuid.uuid4())
            person_ids.add(person_id)

            # Create and add a tracker for the detected person
            tracker = create_tracker(frame, (x, y, w, h))
            trackers.append((tracker, person_id))

    # Update all trackers and remove those that are no longer tracking
    new_trackers = []
    for tracker, person_id in trackers:
        success, bbox = tracker.update(frame)
        if success:
            new_trackers.append((tracker, person_id))
    trackers = new_trackers

    # Draw bounding boxes and IDs on the frame
    for tracker, person_id in trackers:
        success, bbox = tracker.update(frame)
        if success:
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
            cv2.putText(frame, person_id, (p1[0], p1[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('Frame', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

# Print the total number of unique people detected
print(f"Total number of unique people detected: {len(person_ids)}")


4.9.0

0: 384x640 13 persons, 1 boat, 273.9ms
Speed: 2.5ms preprocess, 273.9ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)


AttributeError: module 'cv2' has no attribute 'legacy'